# 开发者指南

本项目的目标是尽量少用三方包，更多的使用 Python 内置的模块和包，设计一个通用的图形库。下面将详细介绍已经设计好的模块和包。

## 项目概览

本项目以 `graph_tensor` 作为包名称，可直接使用 `pip` 进行安装。当前 `graph_tensor` 包含如下结构：

- graph
    - atom.py
        - Meta
        - Drawing
        - TrajectoryDrawing
        - Graph
        - GraphScrollable
    - creator.py
        - Selector
        - SelectorFrame
    - window.py
        - WindowMeta
        - Window
    - param.py
        - Param
    - test.py：记录一些测试案例。
- tool
    - tool_tip.py
        - ToolTip：提示窗口类

下面逐一介绍这些类。

## 1 Meta 是 graph 包的基础类

先进入到包内部：

In [1]:
cd graph_tensor

D:\xinet\Graph\graph_tensor


载入 Meta 类及其测试函数：

In [2]:
from graph.atom import Meta
from graph.test import test_Meta

调用 `test_Meta()` 函数便可以展现 `Meta` 的实现效果。其实，`Meta` 是继承自 `tkinter` 的 `Canvas` 类的，它主要的改进之处在于：将画图的函数定制了一个统一的接口函数：

In [3]:
help(Meta.draw_graph)

Help on function draw_graph in module graph.atom:

draw_graph(self, graph_type, direction, color='blue', width=1, tags=None, **kwargs)
    Draw basic graphic elements.
    
    :param direction: Specifies the orientation of the graphic element. 
        Union[int, float] -> (x_0,y_0,x_,y_1), (x_0, y_0) refers to the starting point of 
        the reference brush (i.e., the left mouse button is pressed), and (x_1, y_1) refers to 
        the end position of the reference brush (i.e., release the left mouse button).
        Multipoint sequences are supported for 'line' and 'polygon',
         for example ((x_0, y_0), (x_1, y_1), (x_2, y_2)).
    :param graph_type: Types of graphic elements.
        (str) 'rectangle', 'oval', 'line', 'arc'(That is, segment), 'polygon'.
        Note that 'line' can no longer pass in the parameter 'fill', and 
        the remaining graph_type cannot pass in the parameter 'outline'.
    :param color: The color of the graphic element.
    :param width: The wi

注意：`color`、`width` 分别表示的是图形的轮廓的颜色和宽度。下面将介绍画图工具的相关库。

## 2 Selector 与 SelectorFrame 用于选择图形的颜色和形状

In [4]:
from graph.creator import Selector, SelectorFrame
from tkinter import Tk

`Selector` （是 `Meta` 的子类）主要设计了画图工具的可供选择的画笔的颜色与绘制的图形的形状。可以使用如下方法进行参考：

In [5]:
root = Tk()
select = Selector(root)
select.grid()
root.mainloop()

`SelectorFrame` 是 `ttk.Frame` 的子类，为 `Selector` 提供了鼠标事件：显示用户选择的画笔的颜色与绘制的图形的形状。使用方法参考：

In [6]:
root = Tk()
select = SelectorFrame(root)
select.grid()
select.layout() # 用于布局 SelectorFrame 的内部小部件
root.mainloop()

Selector 与 SelectorFrame 是面向开发者的，不需要用户调用。

## 3 Drawing 提供了一个使用鼠标画图的工具

`Drawing` 是 `Meta` 的子类，提供了一个使用鼠标画图的接口。点击鼠标左键开始作画，释放鼠标左键完成作画。同样，使用 `test_Drawing()` 可以简单的测试 Drawing 的使用。

In [7]:
from graph.atom import Drawing, TrajectoryDrawing
from graph.test import test_Drawing

`TrajectoryDrawing` 是 `Drawing` 的子类。点击鼠标左键开始作画，移动鼠标左键 `after_time` 后完成作画。

In [8]:
from graph.creator import SelectorFrame
from tkinter import Tk
root = Tk()
selector = SelectorFrame(root)

meta = TrajectoryDrawing(root, selector, after_time=370, background='lightgray')
# Makes the master widget change as the canvas size
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)
meta.layout()
selector.layout()
selector.grid(row=0, column=1, sticky='nwes')

root.mainloop()

## 4 Graph 是可操作图形对象的画图工具

Graph 是 Drawing 的子类。添加了对图形对象的移动、删除操作。

In [9]:
from graph.atom import Graph

In [10]:
def test_GraphWindow1():
    from graph.creator import SelectorFrame
    from graph.tk_utils import Window
    # 设定了图像标注工具的工具栏窗口
    root = Window()
    root.geometry('800x600')
    selector = SelectorFrame(root)
    graph = Graph(root, selector, background='lightgray')
    root.update_edit_menu(graph)
    root.update_file_menu(graph)
    # Makes the master widget change as the canvas size
    graph.layout()
    selector.layout()
    selector.pack(side='right', fill='y')
    root.mainloop()

In [11]:
test_GraphWindow1()

在菜单栏的 File 选项中的 `Seek filename` 可以载入图片。在菜单栏的 Edit 选项中的 move、delete、drawing 可以移动、删除、画出图形。

`GraphScrollable` 是 `Graph` 的子类，增加了图片的滚动条，且设定图片不可移动。

In [12]:
from graph.test import test_GraphWindow

test_GraphWindow()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 2770, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "D:\xinet\Graph\graph_tensor\graph\tk_utils.py", line 86, in <lambda>
    label="Seek filename", command=lambda : self.load_image(graph))
  File "D:\xinet\Graph\graph_tensor\graph\tk_utils.py", line 81, in load_image
    self.update_image()
  File "D:\xinet\Graph\graph_tensor\graph\tk_utils.py", line 74, in update_image
    self.I = self.name2image(self.file_name)
  File "D:\xinet\Graph\graph_tensor\graph\tk_utils.py", line 48, in name2image
    I = Image.open(image_name)
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 2772, in ope